### Pairwise Evaluation
* 두 개 이상의 LLM 생성물 서로 비교한다.

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def evaluate_pairwise(runs: list, example) -> dict:

    # 점수 저장
    scores = {}
    for i, run in enumerate(runs):
        scores[run.id] = i
    

    # 각 예제에 대한 실행 쌍
    answer_a = runs[0].outputs["answer"]
    answer_b = runs[1].outputs["answer"]
    question = example.inputs["question"]

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    grade_prompt = PromptTemplate.from_template(
        """
        You are an LLM judge. Compare the following two answers to a question and determine which one is better.
        Better answer is the one that is more detailed and informative.
        If the answer is not related to the question, it is not a good answer.

        
        # Question:
        {question}
        
        #Answer A: 
        {answer_a}
        
        #Answer B: 
        {answer_b}
        
        Output should be either `A` or `B`. Pick the answer that is better.
        
        #Preference:
        """
    )
    answer_grader = grade_prompt | llm | StrOutputParser()

    score = answer_grader.invoke(
        {
            "question": question,
            "answer_a": answer_a,
            "answer_b": answer_b
        }
    )

    if score == "A": # A가 더 답변을 잘했다.
        scores[runs[0].id] = 1
        scores[runs[1].id] = 0
    elif score == "B": # B가 더 답변을 잘했다.
        scores[runs[0].id] = 0
        scores[runs[1].id] = 1
    else:
        scores[runs[0].id] = 0
        scores[runs[1].id] = 0
        
    return {"key": "ranked_preference", "scores": scores}

In [3]:
from rag import PDFRAG
from langchain_openai import ChatOpenAI

def ask_question_with_llm(llm):

    rag = PDFRAG(
        "data/snow-white.pdf",
        llm
    )

    retriever = rag.create_retriever()

    rag_chain = rag.create_chain(retriever)

    def _ask_question(inputs: dict):
        context = retriever.invoke(inputs["question"])
        context = "\n".join([doc.page_content for doc in context])
        return {
            "question": inputs["question"],
            "context": context,
            "answer": rag_chain.invoke(inputs["question"])
        }
    return _ask_question

In [4]:
from langchain_openai import ChatOpenAI

gpt3 = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

gpt3.invoke("안녕하세요?")

AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 13, 'total_tokens': 34, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6f323598-e81f-4fc4-9ef4-84f870d4b9bc-0', usage_metadata={'input_tokens': 13, 'output_tokens': 21, 'total_tokens': 34, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [ ]:
!pip install langchain-ollama


* Ollama 사용 시 참고

In [13]:
from langchain_ollama import ChatOllama

# Ollama 모델을 불러옵니다.
ollama = ChatOllama(model="llama2")

# Ollama 모델 호출
ollama.invoke("안녕하세요?")

AIMessage(content='\n*giggles* Hello there! *adjusts glasses* How are you today? 😊', additional_kwargs={}, response_metadata={'model': 'llama2', 'created_at': '2024-11-04T05:24:00.4778809Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 20745064300, 'load_duration': 17598162400, 'prompt_eval_count': 28, 'prompt_eval_duration': 1226997000, 'eval_count': 27, 'eval_duration': 1911260000}, id='run-e2fa8fa4-9fb2-450e-904b-f91f7bb28feb-0', usage_metadata={'input_tokens': 28, 'output_tokens': 27, 'total_tokens': 55})

In [6]:
gpt4o_chain = ask_question_with_llm(ChatOpenAI(model="gpt-4o-mini", temperature=0))
gpt3_chain = ask_question_with_llm(ChatOpenAI(model="gpt-3.5-turbo", temperature=0))

# ollama 사용시
# ollama_chain = ask_question_with_llm(ChatOllama(model=""))

In [7]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

cot_qa_evaluator = LangChainStringEvaluator(
    "cot_qa",
    config={"llm": ChatOpenAI(model="gpt-4o-mini", temperature=0)}, # 평가자
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": run.outputs["context"],
        "input": example.inputs["question"]
    }
)

dataset_name = "RAG_EVALUATION_DATASET"

experiment_result1 = evaluate(
    gpt3_chain,
    data=dataset_name,
    evaluators=[cot_qa_evaluator],
    experiment_prefix="MODEL_COMPARE_EVALUATION",
    metadata={
        "variant": "GPT-3.5-turbo 평가 (cot_qa)"
    }
)

experiment_result2 = evaluate(
    gpt4o_chain,
    data=dataset_name,
    evaluators=[cot_qa_evaluator],
    experiment_prefix="MODEL_COMPARE_EVALUATION",
    metadata={
        "variant": "GPT-4o-mini 평가 (cot_qa)"
    }
)

View the evaluation results for experiment: 'MODEL_COMPARE_EVALUATION-af49e507' at:
https://smith.langchain.com/o/c14fe90d-ea29-4834-be1a-fc0bb9b33569/datasets/ad5c603b-00a3-4aa6-9532-972ff3510fb3/compare?selectedSessions=08e7f836-614c-4f51-a55e-ce2bc567a7b9




0it [00:00, ?it/s]

View the evaluation results for experiment: 'MODEL_COMPARE_EVALUATION-c2a4a0dd' at:
https://smith.langchain.com/o/c14fe90d-ea29-4834-be1a-fc0bb9b33569/datasets/ad5c603b-00a3-4aa6-9532-972ff3510fb3/compare?selectedSessions=b81800d3-bb97-42cf-b6ef-024a276beafd




0it [00:00, ?it/s]

In [9]:
from langsmith.evaluation import evaluate_comparative

evaluate_comparative(
    ["MODEL_COMPARE_EVALUATION-0298a9fe", "MODEL_COMPARE_EVALUATION-c2a4a0dd"],
    # 평가자
    evaluators=[evaluate_pairwise]
)

View the pairwise evaluation results at:
https://smith.langchain.com/o/c14fe90d-ea29-4834-be1a-fc0bb9b33569/datasets/ad5c603b-00a3-4aa6-9532-972ff3510fb3/compare?selectedSessions=eca96389-f301-434b-ac28-aa5d4d09cf9e%2Cb81800d3-bb97-42cf-b6ef-024a276beafd&comparativeExperiment=bcdf3a88-2fe7-4400-beb5-b9e277150a60




  0%|          | 0/5 [00:00<?, ?it/s]